In [1]:
!pip install bs4

In [2]:
!pip install requests

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import warnings

In [5]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:97.0) Gecko/20100101 Firefox/97.0',
    'Accept': 'application/json',
}

data = []
page = 0
count = 16

In [6]:
warnings.filterwarnings("ignore", category=requests.packages.urllib3.exceptions.InsecureRequestWarning)

article_number = 0

while article_number < 16000:
    url = f"https://www.dhakatribune.com/api/theme_engine/get_ajax_contents?widget=704&start={page}&count={count}&page_id=1094&subpage_id=0&author=0&tags=&archive_time=&filter="

    try:
        response = requests.get(url, headers=headers, verify=False)
        response.raise_for_status()

        soup = BeautifulSoup(response.json().get("html", ""), 'html.parser')

        articles = soup.select('div.each.col_in')

        if not articles:
            break

        for article in articles:
            title = article.find('h2', class_='title').text.strip()
            link = f"https:{article.find('a', class_='link_overlay')['href']}"

            content_response = requests.get(link, headers=headers, verify=False)
            content_response.raise_for_status()

            content_soup = BeautifulSoup(content_response.content, 'html.parser')

            summary = content_soup.find('div', class_='content_highlights jw_detail_content_holder content mb16')
            summary_text = '. '.join([item.text.strip() for item in summary.find_all(['p', 'li'])]) if summary else None

            article_body = content_soup.find('div', class_='jw_article_body')
            article_text = ' '.join([p.text.strip() for p in article_body.find_all('p')]) if article_body else None

            article_number += 1

            if summary_text is not None or article_text is not None:
                data.append({'URL': link, 'Headline': title, 'Summary': summary_text, 'Article': article_text})
                print(f"Article {article_number}: {link}")

            if article_number >= 16000:
                break

        page += count

    except requests.exceptions.RequestException as err:
        print(f"An error occurred: {err}")
        break

df = pd.DataFrame(data)
df.to_csv('/content/drive/MyDrive/dhaka_tribune_english.csv', index=False, encoding='utf-8')

Streaming output truncated to the last 5000 lines.
Article 11001: https://www.dhakatribune.com/business/244343/%E2%80%98sustained-tariff-policy-can-strengthen
Article 11002: https://www.dhakatribune.com/business/244340/pre-budget-fbcci-proposes-abolishing-advanced
Article 11003: https://www.dhakatribune.com/business/244331/melons-rot-destroyed-as-buyers-decline-during
Article 11004: https://www.dhakatribune.com/business/economy/244328/govt-launches-7.5m-covid-19-response-fund-for
Article 11005: https://www.dhakatribune.com/business/244325/anti-tobacco-groups-urge-raising-taxes-fines-for
Article 11006: https://www.dhakatribune.com/business/banks/244323/bb-server-issues-is-wreaking-havoc-for-bank
Article 11007: https://www.dhakatribune.com/business/244287/tk6-trillion-budget-in-the-works-for-2021-22-govt
Article 11008: https://www.dhakatribune.com/business/244281/apex-md-tax-system-not-business-friendly-want-to
Article 11009: https://www.dhakatribune.com/business/244264/op-ed-addressing-